<a href="https://colab.research.google.com/github/sportlosos/altyn/blob/main/Copy_of_1_LaiLai_Cut_Get_transcripts_json_claude_v_2_6_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (!!)1. Run and restart this cell first

In [ ]:
!pip install git+https://github.com/m-bain/whisperx.git
!pip install git+https://github.com/m-bain/whisperx.git --upgrade
!pip install lightning_fabric
#!!! you must run this cell, wait untill the system asks you to restart, then run again and proceed to the next cell

In [ ]:
!pip install ctranslate2==4.4.0

# (!) 2 Mount G_DRIVE, agree for everything

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# (!:) 3 Get API keys from pyannote huggingface . It's free. You only need to do it once for your account  

In [ ]:
# log in on Huggingface hub (where pretrained pyannote models are hosted)
from huggingface_hub import login
from google.colab import userdata
password = userdata.get('HFpyannote1') # В левой панели есть значок ключа, нужно взять на Huggingface ключ и назвать его HFpyannote1
login(password)

# (!) 4 Provide a path to course folder in line 5
 (:)Checking for audio sourse (for our lectures it's usually m4a, if you have different audio/video file format just (!) specify in line 21 ) Here you can find naming instructions for root dir, enclosing folder, videos etc https://docs.google.com/document/d/1_gZN3tDd3JB7WoKA8wNopDWpDy29ReKaZwa-nNC5Tng/edit  

In [ ]:
#@markdown # 🎵 Audio File Scanner
#@markdown Enter the root directory path and file extension for your audio files, then run this cell.
#@markdown ---
#@markdown **Root Directory Path:**
root_dir = "/content/drive/MyDrive/Folder" #@param {type:"string"}
#@markdown **File Extension (include the dot):**
file_extension = ".mp4" #@param {type:"string"}
#@markdown ---

import os

# Find all subdirectories in the root directory
subdirectories = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

# Initialize an empty list to store the paths of all matching files
audio_files = []

# Loop through each subdirectory
for subdir in subdirectories:
    # Get a list of all files in the current subdirectory
    files = os.listdir(subdir)

    # Loop through each file in the current subdirectory
    for file in files:
        # Check if the file ends with the specified extension
        if file.endswith(file_extension):
            # Add the path of the file to the list
            audio_files.append(os.path.join(subdir, file))

file_count = len(audio_files)

# Print the results
print(f"🔍 **Scan Results:**")
print(f"- Number of {file_extension} files found: {file_count}")
print(f"- Root directory: {root_dir}")
print("\n📁 **File Paths:**")
for path in audio_files:
    print(f"- {path}")

# To use the audio_files list in subsequent cells:
# You can now use the 'audio_files' variable, which contains all the file paths

# (:) 5 Getting mp3 files from whatever format - m4a, mp4, mov, wav  - any will do
- pyannote wants MP3

In [ ]:
import os

# Ensure ffmpeg is available
#!apt install ffmpeg

# Function to convert audio or video to MP3
def convert_to_mp3(input_file):
    output_file = os.path.splitext(input_file)[0] + '.mp3'
    # Use os.system for ffmpeg command execution
    os.system(f'ffmpeg -i "{input_file}" -codec:a libmp3lame -qscale:a 2 "{output_file}"')
    print("Conversion successful. Saved as:", output_file)
    return output_file

# Use the audio_files list from the previous cell
for audio_file in audio_files:  # Changed 'm4a_files' to 'audio_files'
    # Check the file extension and act accordingly
    file_extension = os.path.splitext(audio_file)[-1].lower()
    supported_formats = ['.mp3', '.m4a', '.mp4', '.mov', '.wav']

    if file_extension != '.mp3':
        if file_extension in supported_formats:
            audio_file = convert_to_mp3(input_file=audio_file)  # Convert and update audio_file to the MP3 path
        else:
            print("Unsupported file format.")
    else:
        print("The file is already in MP3 format.")

# (:) 6 Creating a list to store mp3 files

In [ ]:
#
import os

# Initialize an empty list to store the paths of all .m4a files
mp3_files = []

# Loop through each subdirectory
for subdir in subdirectories:
    # Get a list of all files in the current subdirectory
    files = os.listdir(subdir)

    # Loop through each file in the current subdirectory
    for file in files:
        # Check if the file ends with ".m4a"
        if file.endswith(".mp3"):
            # Add the path of the file to the list
            mp3_files.append(os.path.join(subdir, file))

# Print the list of all .m4a files
print(mp3_files)

# (:) 7 STARTING  Whisper and Pyannote to get transcription with word allignment and speaker diarisation,  (Time consuming, 1 hour of audio could take 5 to 20 min.)
Обработка курса из восьми лекций займет больше часа.  


In [ ]:
import whisperx
import gc
import os
import csv
import json
import pandas as pd
import datetime

# Setup and model load
device = "cuda"
batch_size = 16
compute_type = "float16"

# Load model and audio, transcribe

for audio_file in mp3_files:
    model = whisperx.load_model("medium", device, compute_type=compute_type)
    audio = whisperx.load_audio(audio_file)

    result = model.transcribe(audio, batch_size=batch_size)
    print("Initial transcription:", result["segments"])

# Alignment and Diarization
    model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
    result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)
    diarize_model = whisperx.DiarizationPipeline(device=device)
    diarize_segments = diarize_model(audio)
    result = whisperx.assign_word_speakers(diarize_segments, result)
    print("Diarized segments with speaker labels:", result["segments"])

# Save results to JSON
    with open(f"{audio_file}_transcription.json", 'w', encoding='utf-8') as file:
        json.dump(result["segments"], file, ensure_ascii=False, indent=4)

    # Convert JSON data to CSV
    with open(f"{audio_file}_transcription.json", 'r', encoding='utf-8') as file:
        data = json.load(file)

    flat_data = []
    for segment in data:
        segment_start = segment.get('start', 'Unknown')
        segment_end = segment.get('end', 'Unknown')
        segment_text = segment.get('text', '')
        segment_speaker = segment.get('speaker', 'Unknown')

        if 'words' in segment and segment['words']:  # Ensure 'words' is not only present but also not empty
            for word in segment['words']:
                flat_data.append({
                    'segment_start': segment_start,
                    'segment_end': segment_end,
                    'segment_text': segment_text,
                    'segment_speaker': segment_speaker,
                    'word': word.get('word', 'Unknown'),
                    'word_start': word.get('start', 'Unknown'),
                    'word_end': word.get('end', 'Unknown'),
                    'word_score': word.get('score', 'Unknown'),
                    'word_speaker': word.get('speaker', segment_speaker)
                })
        else:
            # If there are no words, add a row representing the segment without word details
            flat_data.append({
                'segment_start': segment_start,
                'segment_end': segment_end,
                'segment_text': segment_text,
                'segment_speaker': segment_speaker,
                'word': 'No words',
                'word_start': 'N/A',
                'word_end': 'N/A',
                'word_score': 'N/A',
                'word_speaker': 'N/A'
            })


    df = pd.DataFrame(flat_data)
    df.to_csv(f"{audio_file}_transcription.csv", index=False)

    print(f"{audio_file}_transcription.csv has been successfully saved")

**Go to stage `*`2.Bunch Clean for transcripts_Lai-Lai-Cut_v2.3 `*`
Переходите к этапу очистки от слов паразитов (ваша принцесса в другом замке) **

`CODE`
* [Folder with clean copies](https://drive.google.com/drive/folders/186jZV_42xPERO6AjjorpctJu-5UZxkBt?usp=sharing ) of Lai-Lai-cut_v2.3 4 stages of automation
* [Папка с чистыми копиями](https://drive.google.com/drive/folders/186jZV_42xPERO6AjjorpctJu-5UZxkBt?usp=sharing ) Lai-Lai-Cut_v.2.3  - 4 этапа обработки видео.

-------------
`INSTRUCTIONS`
* [Instructions ](https://docs.google.com/document/d/1_gZN3tDd3JB7WoKA8wNopDWpDy29ReKaZwa-nNC5Tng/edit) for naming and bunch processing. Details on input and output file names  for every stage.   
*[Файл с инструкциями](https://docs.google.com/document/d/1_gZN3tDd3JB7WoKA8wNopDWpDy29ReKaZwa-nNC5Tng/edit) к номенклатуре файлов, и к работе с четырьмя этапами, в
котором можно уточнить названия файлов ввода и вывода   

# **`! Disconnect GPU time and proceed to stage 2.`**